<a href="https://colab.research.google.com/github/kasier48/DeepLearning/blob/main/Pratice_Week5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [5주차] 심화과제: 다양한 형태의 입력을 가지는 LLM 서비스 개발

코드 리뷰 LLM 서비스는 다음과 같이 구성됩니다:

- **입력:**  코드 리뷰 서비스의 입력은 다음 두 가지 중 하나로 결정하시면 됩니다:
    - **하나의 함수와 같이 짧은 코드:** 사용자에게 직접 코드 리뷰를 받고 싶은 코드를 입력으로 받습니다. 함수 구현과 같이 짧은 코드가 될 것 입니다.
    - **Github repository:** 특정 github repository 링크를 입력받아, 프로젝트의 모든 코드들을 리뷰 받을 수도 있습니다.
- **방법론:** 입력 형태에 따라 다음 두 가지 방법 중 하나를 선택하시면 됩니다:
    - **짧은 코드:** 만약 짧은 길이의 코드를 입력받는 경우, 간단한 prompting을 통해 LLM만으로 코드 리뷰를 받으시면 됩니다.
    - **Github repository:** repository를 입력으로 받는 경우, 모든 코드들을 concat하여 짧은 코드와 같이 prompting 할 수도 있습니다. 아니면 RAG를 통해 문제 있는 부분들만 추출한 후(이 경우 prompt를 어떻게 정할지가 중요합니다), 이들을 가지고 코드 리뷰를 진행할 수도 있습니다.
- **출력:** 주어진 코드에서 문제되는 부분과 왜 문제가 되는지에 대한 간단한 설명이 출력되어야 합니다.

위의 LLM 서비스를 다음 요구사항을 충족시키며 개발하시면 됩니다:

- [ ]  입력 및 방법론 결정
    - 위에서 주어진 선택지들 중에서 어떤 입력을 받으실지 정하셔야 합니다.
- [ ]  코드 리뷰 prompting 제안
    - 코드 리뷰가 출력되도록 prompt를 구성하셔야 합니다. 만약 RAG도 활용하셔야 하는 경우, 문제 있는 부분들을 우선 추출할 수 있도록 prompt를 설계하는 것이 중요할 것입니다.
- [ ]  실제 코드 리뷰 결과 출력

In [9]:
!pip install langchain-community langchain-chroma langchain-openai bs4

In [10]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from google.colab import userdata
import os, requests

# [MYCODE] gpt-4o-mini로 설정
openai_key = userdata.get('openai_key')
llm = ChatOpenAI(model="gpt-4o-mini", api_key=openai_key)

# [MYCODE] git hun raw url을 활용하여 코드의 내용을 가져오도록 함.
raw_url = "https://raw.githubusercontent.com/kasier48/DeepLearning/main/Week5/test.py"

response = requests.get(raw_url)
if response.status_code == 200:
    code_content = response.text
    print("코드 내용을 성공적으로 가져왔습니다.")
    print(code_content[:500])
else:
    print(f"파일을 가져오는 데 실패했습니다. 상태 코드: {response.status_code}")
    print(response.text)

# [MYCODE] 문서 객체를 생성하고 토큰화 한다.
docs = [Document(page_content=code_content)]

for idx, doc in enumerate(docs):
  print(f"idx; {idx + 1}, content: {doc.page_content}")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(api_key=openai_key)
)

retriever = vectorstore.as_retriever()

# [MYOCDE] User prompt를 하여 lang chain으로 실행시킨다.
user_msg = "코드를 리뷰하고 문제점을 말해줘."

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
prompt = hub.pull("rlm/rag-prompt")

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
print(rag_chain.invoke(user_msg))

코드 내용을 성공적으로 가져왔습니다.
def Devide(a, b):
  return a / b

a = 10
b = 0

result = Devide(a, b)
print(result)
idx; 1, content: def Devide(a, b):
  return a / b

a = 10
b = 0

result = Devide(a, b)
print(result)


/usr/local/lib/python3.10/dist-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


코드에서 문제가 되는 부분은 b가 0일 때 0으로 나누는 연산을 시도하는 것입니다. 이 경우 ZeroDivisionError가 발생하므로, b가 0이 아닐 때만 나누기 연산을 수행하도록 조건을 추가해야 합니다. 예를 들어, b가 0일 경우 적절한 에러 메시지를 출력하는 방법이 좋습니다.
